In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import json
import os,sys
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

In [5]:
pd.options.display.float_format = '{:,.2f}'.format
dataset_base_dir = '../datasets/students-dataset/'
train_file = 'benchmark/dataset.csv'

In [8]:
with open(os.path.join(dataset_base_dir, train_file)) as f:
    train_df = pd.read_csv(f, low_memory=False)
    train_df = train_df.set_index('id_questionnaire')

train_df

,id_student_original,id_year,id_grade,id_class_group,id_school,id_student_16_19,id_school_16_19,score_MAT,level_MAT,score_LEN,...,t_extent_of_student_involvement_during_class,t_extent_of_teaching_methods_variety,t_extent_of_opinion_on_school,t_extent_of_class_behaviour,t_extent_of_resource_variety,t_extent_of_good_work_by_non_teachers,t_number_of_individual_training_topics,t_number_of_subjects_taught,t_extent_of_satisfaction_job_and_school,t_extent_of_results_satisfaction
id_questionnaire,,,,,,,,,,,,,,,,,,,,,
1,"33,613.00",2016,3,A,"2,415.00","14,374.00",569.00,564.87,3.00,535.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"19,294.00",2016,3,A,"1,842.00","8,001.00",273.00,388.34,1.00,293.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"19,587.00",2016,3,NaN,"1,432.00","8,142.00",82.00,386.59,1.00,514.81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"29,985.00",2016,3,A,"2,280.00","12,800.00",505.00,487.76,2.00,449.25,...,0.56,0.67,1.00,0.50,0.46,0.56,NaN,NaN,NaN,0.67
5,"6,982.00",2016,3,A,"2,040.00","2,606.00",390.00,709.79,4.00,598.72,...,0.67,0.73,1.00,0.83,0.62,0.89,1.00,NaN,NaN,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83853,"2,500.00",2019,6,B,"1,278.00",NaN,271.00,400.86,2.00,446.65,...,0.56,0.80,1.00,0.50,0.42,1.00,1.00,NaN,1.00,0.17
83854,"5,784.00",2019,6,A,"1,079.00",NaN,78.00,597.02,3.00,632.60,...,0.67,0.73,0.67,0.67,0.50,1.00,1.00,NaN,0.72,0.50
83855,"7,708.00",2019,6,X,"1,486.00","3,036.00",459.00,707.93,4.00,400.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
print("Size of the dataset: ", len(train_df))
print("Number of features: ", len(train_df.columns))

Size of the dataset:  80174
Number of features:  159


In [15]:
# Get protected attribute gender
features_protected= train_df["s_gender"]
print("Features: ", features_protected)

Features:  id_questionnaire
1        FEMALE
2          MALE
3        FEMALE
4          MALE
5        FEMALE
          ...  
83853    FEMALE
83854    FEMALE
83855      MALE
83856    FEMALE
83857      MALE
Name: s_gender, Length: 80174, dtype: object


In [17]:
# Distribution of the protected attribute gender
print("Distribution of the protected attribute")
print(features_protected.value_counts())
print("Percentage of the protected attribute")
print(features_protected.value_counts(normalize=True))

Distribution of the protected attribute
MALE      39343
FEMALE    37881
Name: s_gender, dtype: int64
Percentage of the protected attribute
MALE     0.51
FEMALE   0.49
Name: s_gender, dtype: float64


In [22]:
# Distribution of the protected attribute birth country 
print(train_df["s_birth_country"].isna().sum())
print(train_df["s_birth_country"].value_counts())
print(train_df["s_birth_country"].value_counts(normalize=True))

0
ESP    15731
VEN      253
ITA      153
CUB      145
COL      119
       ...  
CRI        1
DNK        1
IRN        1
GEO        1
BLR        1
Name: s_birth_country, Length: 73, dtype: int64
ESP   0.91
VEN   0.01
ITA   0.01
CUB   0.01
COL   0.01
      ... 
CRI   0.00
DNK   0.00
IRN   0.00
GEO   0.00
BLR   0.00
Name: s_birth_country, Length: 73, dtype: float64


In [24]:
# Drop the rows in which the protected attribute birth country is missing
train_df = train_df.dropna(subset=["s_birth_country"])
print("Size of the dataset after dropping missing values: ", len(train_df))

Size of the dataset after dropping missing values:  17313


In [27]:
# Distribution of the protected attributes gender x birth country
for country in train_df["s_birth_country"].unique():
    print("Distribution of the protected attribute gender for birth country: ", country)
    print(train_df[train_df["s_birth_country"]==country]["s_gender"].value_counts())
    print("Percentage of the protected attribute:\n", train_df[train_df["s_birth_country"]==country]["s_gender"].value_counts(normalize=True))

Distribution of the protected attribute gender for birth country:  ESP
MALE      7973
FEMALE    7758
Name: s_gender, dtype: int64
Percentage of the protected attribute:
 MALE     0.51
FEMALE   0.49
Name: s_gender, dtype: float64
Distribution of the protected attribute gender for birth country:  ITA
MALE      79
FEMALE    74
Name: s_gender, dtype: int64
Percentage of the protected attribute:
 MALE     0.52
FEMALE   0.48
Name: s_gender, dtype: float64
Distribution of the protected attribute gender for birth country:  VEN
FEMALE    129
MALE      124
Name: s_gender, dtype: int64
Percentage of the protected attribute:
 FEMALE   0.51
MALE     0.49
Name: s_gender, dtype: float64
Distribution of the protected attribute gender for birth country:  PHL
FEMALE    8
MALE      8
Name: s_gender, dtype: int64
Percentage of the protected attribute:
 FEMALE   0.50
MALE     0.50
Name: s_gender, dtype: float64
Distribution of the protected attribute gender for birth country:  CUB
MALE      75
FEMALE    70